### 给定一个DataFrame，请构造其转置且不得使用“.T”。
### 身体质量指数BMI的计算方式是体重（单位为kg）除以身高（单位为m）的平方，请找出具有最高BMI指数对应同学的姓名。
### 在clip()中，超过边界的只能截断为边界值，如果要把超出边界的替换为自定义的值，可以如何做？
### 在Numpy中也有一个同名函数np.diff()，它与pandas中的diff功能相同吗？请查阅文档说明。
### rolling对象的默认窗口方向都是向下滑动的，某些情况下用户需要逆向滑动的窗口，例如对[1,2,3]设定窗口为2的逆向sum操作，结果为[3,5,NaN]，此时应该如何实现？

**高数中的n阶差分**

In [105]:
# 给定一个DataFrame，请构造其转置且不得使用“.T”。
import pandas as pd
import numpy as np

df = pd.DataFrame({"A": [1,2,3], "B": [4,5,6]}, index=list("abc"))

df2 = pd.DataFrame(data=df.values.T, index=df.columns, columns=df.index)

df.T.equals(df2)

# 身体质量指数BMI的计算方式是体重（单位为kg）除以身高（单位为m）的平方，请找出具有最高BMI指数对应同学的姓名。

df = pd.read_csv('data/learn_pandas.csv')

idxmax = (df.Weight / (df.Height/100) ** 2).idxmax()
df.T[idxmax]['Name']

# 在clip()中，超过边界的只能截断为边界值，如果要把超出边界的替换为自定义的值，可以如何做？

s = pd.Series(np.arange(6))

s.clip(1, 4)

s.where(s >= 1, 999).where(s <= 4, 888)


# 在Numpy中也有一个同名函数np.diff()，它与pandas中的diff功能相同吗？请查阅文档说明。

s = pd.Series([1, 3, 7, 5, 3, 6, 9])
np.diff(s.values, 2) # 指n阶差分
s.diff() # 差值


# rolling对象的默认窗口方向都是向下滑动的，某些情况下用户需要逆向滑动的窗口，例如对[1,2,3]设定窗口为2的逆向sum操作，结果为[3,5,NaN]，此时应该如何实现？

s = pd.Series([1,2,3])

print(s)

rolling = s[::-1].rolling(2).sum()[::-1]

rolling





0    1
1    2
2    3
dtype: int64


0    3.0
1    5.0
2    NaN
dtype: float64

In [184]:
import pandas as pd

"""
在data/ch2/clothing_store.csv中记录了某服装店商品的信息，每件商品都有一级类别（type_1）、二级类别（type_2）、进价（buy_price）、售价（sale_price）和唯一的商品编号（product_id）。

利润指售价与进价之差，求商品的平均利润。

从原表构造一个同长度的Series，索引是商品编号，value中的每个元素是对应位置的商品信息字符串，字符串格式为“商品一级类别为...，二级类别为...，进价和售价分别为...和...。”。

表中有一个商品的二级类别与一级类别明显无法对应，例如一级类别为上衣，但二级类别是拖鞋，请找出这个商品对应的商品编号。

求各二级类别中利润最高的商品编号。

"""
df = pd.read_csv('data/ch2/clothing_store.csv')

# print(df)

# 利润指售价与进价之差，求商品的平均利润。
(df['sale_price'] -  df['buy_price']).mean()

# 从原表构造一个同长度的Series，索引是商品编号，value中的每个元素是对应位置的商品信息字符串，
# 字符串格式为“商品一级类别为...，二级类别为...，进价和售价分别为...和...。”

pd.Series(
  data=[
    '商品一级类别为%s二级类别为%s进价和售价分别为%d,%d'%(type_1, type_2, sale_price, buy_price) 
      for type_1, type_2, sale_price, buy_price in df[['type_1', 'type_2', 'sale_price', 'buy_price']].values
  ]
, index=df.product_id)

pattern = "商品一级类别为{}，二级类别为{}，进价和售价分别为{:d}和{:d}。"

res = df.apply(
    lambda x: pattern.format(*x.values), axis = 1)

res


# 表中有一个商品的二级类别与一级类别明显无法对应，例如一级类别为上衣，但二级类别是拖鞋，请找出这个商品对应的商品编号。

df.drop_duplicates(['type_1', 'type_2'])

# df = df.drop(6021)

# df.tail(9999 - 6010).head(20)

# 求各二级类别中利润最高的商品编号。

temp = df.copy()

temp["profit"] = temp['sale_price'] - temp['buy_price']

temp.sort_values(
  by=["type_2", "profit"], 
  ascending=[True, False]
).drop_duplicates("type_2")[["type_2", "product_id", "profit"]].head(10)




,type_2,product_id,profit
1405,T恤,S009881,49
162,中裤,S005119,49
820,冲锋衣,S009181,49
664,凉鞋,S001114,49
858,拖鞋,S002385,49
492,游泳裤,S009267,49
515,羽绒服,S003205,49
1073,跑鞋,S005340,49
1824,长裤,S005169,49


In [307]:
# 在data/ch2/student_grade.csv中记录了某课程中每位学生学习情况，包含了学生编号、期中考试分数、期末考试分数、
# 回答问题次数和缺勤次数。
# 请注意，在本题中仅允许使用本章中出现过的函数，不得使用后续章节介绍的功能或函数（例如loc和pd.cut()），但读者可在学习完后续章节后，自行再给出基于其他方案的解答

df = pd.read_csv("data/ch2/student_grade.csv")

# print(df)

# 求出在缺勤次数最少的学生中回答问题次数最多的学生编号。
s = df.sort_values(by=["Question_Answering_Times", "Absence_Times"], ascending=[False, True])['Student_ID']

s[s.index[0]]


# 总评分数为百分之四十的期中考试成绩加百分之六十的期末考试成绩

add = df.Question_Answering_Times.where(df.Question_Answering_Times < 10, 10)

absence_sum = df.Absence_Times * 5

df['sum'] = df.Mid_Term_Grade * 0.4 + df.Final_Grade * 0.6 + add - absence_sum

data = df.where(absence_sum < 5, 0).where(df['sum'] < 100, 80)

data['sum'].head()

# 在表中新增一列“等第”，规定当学生总评低于60分时等第为不及格，总评不低于60分且低于80分时为及格，总评不低于80分且低于90分时为良好，总评不低于90分时为优秀，请统计各个等第的学生比例

df['等第'] = df['sum'].where(df['sum'] > 100, '优秀').where(df['sum'] > 90, '良好').where(df['sum'] > 80, '及格').where(df['sum'] > 60, '不及格')
# .where(df['sum'] >= 90, '优秀')

grade_dict = {0:"不及格", 1:"及格", 2:"良好", 3:"优秀"}


df['grade'] = ((df['sum'] > 90) * 1 + (df['sum'] > 80)* 1  + (df['sum']  > 60)* 1 ).replace(grade_dict)

df.grade.value_counts() / df.Student_ID.count()

# === df.grade.value_counts(normalize=True)

# df.grade.value_counts()


grade
及格     0.510
良好     0.305
优秀     0.105
不及格    0.080
Name: count, dtype: float64